# K-Nearest Neighbors (KNN)
Classifing student success data by means of the [KNeighborsClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) from the sklearn module. The data set comes from UCI's machine learning repository and can be downloaded [here](https://archive.ics.uci.edu/ml/machine-learning-databases/00320/). A description of the data can be found [here](https://archive.ics.uci.edu/ml/datasets/student+performance).

## Import Data
Import the data into a pandas dataframe. Get dummy variables for each categorical predictor in the data set and return the design matirx. Create a normalized and standardized design matrix as well to compare model preformance. Choose the response variable to be only the final grade from the term. Convert response variable to three classes *0 (fail) , 1 (pass),* and *2 (incomplete)*. A student is put into the passing class if they got a score bigger or equal to 10, the failing class if they got a score below 10 but above 0, and the incomplete calss if the student received a 0.

In [1]:
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectPercentile
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils.extmath import cartesian
from sklearn import preprocessing
from sklearn import metrics

df = pd.read_csv('student-por2.csv')
df = pd.get_dummies(df)#, drop_first=True)

def response_conv(arr):
    new = []
    for i in arr:
        if (i > 0 and i < 10):            # condition where student failed
            new.append(0)                 
                                          
        elif (i >= 10):                   # condition where student passed
            new.append(1)                 
    
        else:                             # condition where student received an incomplete
            new.append(2)
    return(new)                           # 1-dimensional array returned


X = df.drop('G3',1)                       # This is the design matrix
y = response_conv(list(df.G3))            # This is the multinomial response vector

select = SelectPercentile()
newX = select.fit_transform(X,y)          # Select most influential predictors

X_scale = preprocessing.scale(newX)       # Scaled design matrix
X_norm = preprocessing.normalize(newX)    # Normalized design matrix

random.seed(42)
X1_train, X1_test, y1_train, y1_test = train_test_split(newX, y, test_size=0.33,random_state=42)
X2_train, X2_test, y2_train, y2_test = train_test_split(X_scale, y, test_size=0.33,random_state=42)
X3_train, X3_test, y3_train, y3_test = train_test_split(X_norm, y, test_size=0.33,random_state=42)

C:\Users\006988889-SA\Downloads\conda\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)
C:\Users\006988889-SA\Downloads\conda\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


## Niave Accuracy
Before we start training and selecting parametrs for our model, we must find the distribution of the classes amongst the response variable. Depnding on which class is the dominate class, our model should preform better than just guessing the dominate class for each observation. For example, if the dominate class is 1 and 1's comprise of 83% of the response data, then our model should have higher than 83% accuracy. 

In [2]:
zero = 0
one = 0
two = 0

for i in y3_test:
    if i == 0:
        zero += 1
    elif i == 1:
        one += 1
    else:
        two += 1

num1 = round((zero/len(y3_test))*100,2)
num2 = round((one/len(y3_test))*100,2)
num3 = round((two/len(y3_test))*100,2)
print("The response vector has the following distribution: \nzeros: %r zeros comprising of %r percent of the response data. \nones: %r ones comprising of %r percent of the response data. \ntwos: %r twos comprising of %r percent of the response data." % (zero,num1,one,num2,two,num3))
print("\n")

The response vector has the following distribution: 
zeros: 23 zeros comprising of 10.7 percent of the response data. 
ones: 187 ones comprising of 86.98 percent of the response data. 
twos: 5 twos comprising of 2.33 percent of the response data.




##  Optimal Model Parameters
Now that we have a baseline accuracy to improve upon we will need to tune the models parameters to be optimal. The fucntion *opt* performs 10-fold cross validation and finds the optimal number of neighbors to be used in the model. Following that, we store the optimal parameters with respect to each design matrix.

In [3]:
start_time = time.time()
myList = list(range(1,50))
neighbors = list(filter(lambda x: x % 2 != 0, myList))
combos = cartesian([['uniform','distance'],neighbors])

def opt(X,y):
    acc = []

    for w, k in combos:
        knn = KNeighborsClassifier(n_neighbors=int(k),weights=str(w))
        scores = cross_val_score(knn, X, y, cv=10, scoring='accuracy')
        acc.append(scores.mean())
    
    #MSE = [1 - x for x in cv_scores]
    opt_ = combos[acc.index(max(acc))]
    return(opt_)

w1, k1 = opt(X1_train,y1_train)
w2, k2 = opt(X2_train,y2_train)
w3, k3 = opt(X3_train,y3_train)

print ("The optimal weight function and number of neighbors is %s and %r respectively for Non-standardized design matrix." % (w1,k1))
print ("The optimal weight function and number of neighbors is %s and %r respectively for standardized design matrix." % (w2,k2))
print ("The optimal weight function and number of neighbors is %s and %r respectively for normalized design matrix." % (w3, k3))
print("Run time: %r minutes" % (round((int(time.time() - start_time)/60),2)))

The optimal weight function and number of neighbors is uniform and '47' respectively for Non-standardized design matrix.
The optimal weight function and number of neighbors is uniform and '3' respectively for standardized design matrix.
The optimal weight function and number of neighbors is uniform and '5' respectively for normalized design matrix.
Run time: 0.05 minutes


## Fit and Predict
Fit the KNN model to each design matrix and create a dataframe comparing each models predictions to the actual value of the test set.

In [4]:
knn1 = KNeighborsClassifier(n_neighbors=int(k1),weights=str(w1)).fit(X1_train,y1_train)
knn2 = KNeighborsClassifier(n_neighbors=int(k2),weights=str(w2)).fit(X2_train,y2_train)
knn3 = KNeighborsClassifier(n_neighbors=int(k3),weights=str(w3)).fit(X3_train,y3_train)

knn_pred1 = knn1.predict(X1_test)
knn_pred2 = knn2.predict(X2_test)
knn_pred3 = knn3.predict(X3_test)

pred = pd.DataFrame(list(zip(y1_test, knn_pred1,knn_pred2,knn_pred3)), columns=['y_act','y_knn','y_knn_stand','y_knn_norm'])
pred.index.name = 'Obs'
pred


,y_act,y_knn,y_knn_stand,y_knn_norm
Obs,,,,
0,1,1,1,1
1,1,1,1,1
2,1,1,1,1
3,1,1,1,1
4,1,1,1,1
5,1,1,1,1
6,1,1,1,1
7,0,1,0,1
8,1,1,1,1


## Results
Returns model accuracay, confusion matrix and classifiaction report for each respective model. While the non-standardized design matrix has the highest accuracy it may not be the best model becasue it apperas to not acknowledge the presence of 2's in the response variable. The normalized design matrix doesn't preform better than our niave accuracy. Thus the standardized matrix should be considered as our final model.

In [5]:
cm_knn1 = pd.DataFrame(metrics.confusion_matrix(y1_test, knn_pred1), index = ['Fail(0)','Pass(1)','Inc(2)'],columns=['Fail(0)','Pass(1)','Inc(2)'])
cm_knn2 = pd.DataFrame(metrics.confusion_matrix(y2_test, knn_pred2), index = ['Fail(0)','Pass(1)','Inc(2)'],columns=['Fail(0)','Pass(1)','Inc(2)'])
cm_knn3 = pd.DataFrame(metrics.confusion_matrix(y3_test, knn_pred3), index = ['Fail(0)','Pass(1)','Inc(2)'],columns=['Fail(0)','Pass(1)','Inc(2)'])

print ("The accuracy of the Non-standarized KNN model is: ", knn1.score(X1_test,y1_test))
print("\n")
print ("The accuracy of the Standardized KNN model is: ", knn2.score(X2_test,y2_test))
print("\n")
print ("The accuracy of the Normalized KNN model is: ", knn3.score(X3_test,y3_test))
print("\n")

print("Non-standarized KNN Confusion Matrix: \n", cm_knn1)
print("\n")
print("Standarized KNN Confusion Matrix: \n", cm_knn2)
print("\n")
print("Normalized KNN Confusion Matrix: \n", cm_knn3)
print("\n")

print("Classification report for Non-standardized design matrix:\n", metrics.classification_report(y1_test,knn_pred1))
print("\n")
print("Classification report for standardized design matrix:\n", metrics.classification_report(y2_test,knn_pred2))
print("\n")
print("Classification report for Normalized design matrix:\n", metrics.classification_report(y3_test,knn_pred3))

The accuracy of the Non-standarized KNN model is:  0.916279069767


The accuracy of the Standardized KNN model is:  0.893023255814


The accuracy of the Normalized KNN model is:  0.86976744186


Non-standarized KNN Confusion Matrix: 
          Fail(0)  Pass(1)  Inc(2)
Fail(0)       16        7       0
Pass(1)        6      181       0
Inc(2)         4        1       0


Standarized KNN Confusion Matrix: 
          Fail(0)  Pass(1)  Inc(2)
Fail(0)       14        8       1
Pass(1)       12      175       0
Inc(2)         1        1       3


Normalized KNN Confusion Matrix: 
          Fail(0)  Pass(1)  Inc(2)
Fail(0)        7       16       0
Pass(1)       10      177       0
Inc(2)         1        1       3


Classification report for Non-standardized design matrix:
              precision    recall  f1-score   support

          0       0.62      0.70      0.65        23
          1       0.96      0.97      0.96       187
          2       0.00      0.00      0.00         5

avg / t

C:\Users\006988889-SA\Downloads\conda\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
